# SMA Crossover Research

Research notebook for the Simple Moving Average crossover strategy.
Uses QuantBook to explore historical data and validate the signal.

In [ ]:
from QuantConnect import *
from QuantConnect.Research import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

qb = QuantBook()

In [ ]:
spy = qb.add_equity("SPY", Resolution.DAILY)
history = qb.history(spy.symbol, 365, Resolution.DAILY)
history.head()

In [ ]:
close = history["close"].unstack(level=0)
close.columns = ["SPY"]

close["SMA_10"] = close["SPY"].rolling(10).mean()
close["SMA_30"] = close["SPY"].rolling(30).mean()

close[["SPY", "SMA_10", "SMA_30"]].plot(
    figsize=(14, 7),
    title="SPY with SMA Crossover Signals",
)
plt.ylabel("Price")
plt.show()

In [ ]:
# Generate crossover signals
close["signal"] = np.where(close["SMA_10"] > close["SMA_30"], 1, 0)
close["position"] = close["signal"].diff()

# Calculate strategy returns
close["returns"] = close["SPY"].pct_change()
close["strategy_returns"] = close["returns"] * close["signal"].shift(1)

cumulative = (1 + close[["returns", "strategy_returns"]].dropna()).cumprod()
cumulative.columns = ["Buy & Hold", "SMA Crossover"]
cumulative.plot(figsize=(14, 7), title="Strategy vs Buy & Hold")
plt.ylabel("Cumulative Returns")
plt.show()